### 386 Project 

Cleaning and organizing and requesting information to build my cleaned Data Frame. 

Lucy Storts 

Stat 386 - BYU - Dr. Tass

I found a bunch of data from https://www.bts.gov/ and https://www.faa.gov/airports available for download. 

My goal with these data sets is to create variables I can merge together for my own data set. 

First thing: read in and preview each file...

All the data files are .xlsx or .xls (except for the one pdf?) and can be found at: /Users/lucystorts/Desktop/BYU/Stat386/386Project/PotentialData

### Libraries

In [1]:
import pandas as pd
import numpy as np
import requests
import re
import urllib.parse
import matplotlib.pyplot as plt             
import seaborn as sns
import json
from datetime import datetime

In [2]:
partialpath = '/Users/lucystorts/Desktop/BYU/Stat386/386Project/PotentialData/'

In [3]:
# !pip install xlrd

# !conda install -c anaconda xlrd --yes


### Notes on data: 

**rankings** 
- U.S. Airports, ranked by 2022 Systemwide* Scheduled Enplanements Passenger numbers in millions (000,000)
- https://www.bts.gov/topics/airlines-and-airports/airport-rankings-2022

**OriginatingPassengers**
- U.S. Airports ranked by 2022 Originating Domestic Passengers  
- Source: Bureau of Transportation Statistics, Origin & Destination Survey
- DB1B Ticket, Based on 10 Percent Ticket Sample
- O&D numbers are not comparable to T-100 Market Enplanement numbers
- 

**airportArrivalsOnTimeCurrentMonthly**
- Table 3
- Ranking of Major Airport On-Time Arrival Performance in for Sept 22 - Aug 23					
- Based on Marketing Carrier Performance, (Percent On-Time)					

**airportArrivalsOnTimeHistorical**
- Table 4
- Ranking of Major Airport On-Time Arrival Performance Year-to-date through December
- tables for 2004 - 2022						
- Based on Marketing Carrier Performance, (Percent On-Time)		

**airportDepartureOnTimeCurrentMonthly**
- Table 5
- Ranking of Major Airport On-Time Departure Performance for Sept 22 - Aug 23						
- Based on Marketing Carrier Performance, (Percent On-Time)								

In [4]:
rankings = pd.read_excel(partialpath+'AirportRankings2022.xlsx')
originatingPassengers = pd.read_excel(partialpath+'OriginatingPassengers.xlsx')
airportLOCID = pd.read_excel(partialpath+'NPIAS-Report-2019-2023-Appendix-A.xlsx')
airportPFC = pd.read_excel(partialpath+'pfc_airports_20231031_0.xlsx')
airportGrants = pd.read_excel(partialpath+'airportGrants.xlsx')

/Users/lucystorts/opt/anaconda3/envs/stat386/lib/python3.10/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


### Viewing the data

In [5]:
onTimePath = partialpath+'AnnualAirlineOn-TimeRankings2003-2020.xlsx'
excelFile = pd.ExcelFile(onTimePath)
sheetNames = excelFile.sheet_names
airlineArrivalsOnTime = {}
for sheetName in sheetNames:
    airlineArrivalsOnTime[sheetName] = excelFile.parse(sheetName)

# use .keys() to see all tables names!
airlineArrivalsOnTime['2003'].head(3)

,Unnamed: 0,Carrier,On-Time Arrival Pct.
0,1,Hawaiian,90.77
1,2,SkyWest,87.59
2,3,Southwest,86.30


In [6]:
onTimePath = partialpath+'Table3-RankingMajorAirportOn-TimeArrivalPerformanceAugust2023.xlsx'
excelFile = pd.ExcelFile(onTimePath)
sheetNames = excelFile.sheet_names
airportArrivalsOnTimeCurrentMonthly = {}
for sheetName in sheetNames:
    airportArrivalsOnTimeCurrentMonthly[sheetName] = excelFile.parse(sheetName)

# use .keys() to see all tables names!
airportArrivalsOnTimeCurrentMonthly['Aug23'].head(3)

,Rank,2022-08-01 00:00:00,%,Rank.1,2023-08-01 00:00:00,%.1,Unnamed: 6,
0,1,"Salt Lake City, UT (SLC)",84.62,1.0,"Salt Lake City, UT (SLC)",83.87,NaN,NaN
1,2,"Seattle, WA (SEA)",84.40,2.0,"Minneapolis/St. Paul, MN (MSP)",83.74,NaN,NaN
2,3,"Minneapolis/St. Paul, MN (MSP)",83.07,3.0,"Washington, DC (IAD)",82.91,NaN,NaN


In [7]:
onTimePath = partialpath+'Table4RankingMajorAirportOn-TimeArrivalPerformanceYear-to-dateDecember2003-Dec2022.xlsx'
excelFile = pd.ExcelFile(onTimePath)
sheetNames = excelFile.sheet_names
airportArrivalsOnTimeHistorical = {}
for sheetName in sheetNames:
    airportArrivalsOnTimeHistorical[sheetName] = excelFile.parse(sheetName)

# use .keys() to see all tables names!
airportArrivalsOnTimeHistorical['2022'].head(3)

,Rank,"Jan 1 - December 31, 2021",%,Rank.1,"Jan 1 - December 31, 2022",%.1
0,1,"Atlanta, GA (ATL)",87.98,1.0,"Salt Lake City, UT (SLC)",84.32
1,2,"Minneapolis/St. Paul, MN (MSP)",87.52,2.0,"Atlanta, GA (ATL)",82.38
2,3,"Charlotte, NC (CLT)",86.43,3.0,"Minneapolis/St. Paul, MN (MSP)",81.94


In [8]:
onTimePath = partialpath+'Table5-RankingMajorAirportOn-TimeDeparturePerformanceAugust2023.xlsx'
excelFile = pd.ExcelFile(onTimePath)
sheetNames = excelFile.sheet_names
airportDepartureOnTimeCurrentMonthly = {}
for sheetName in sheetNames:
    airportDepartureOnTimeCurrentMonthly[sheetName] = excelFile.parse(sheetName)

airportDepartureOnTimeCurrentMonthly['Aug23'].head(3)

,Rank,2022-08-01 00:00:00,%,Rank.1,2023-08-01 00:00:00,%.1
0,1,"Salt Lake City, UT (SLC)",83.78,1.0,"Salt Lake City, UT (SLC)",82.55
1,2,"San Francisco, CA (SFO)",82.89,2.0,"Minneapolis/St. Paul, MN (MSP)",81.81
2,3,"Detroit, MI (DTW)",82.86,3.0,"Seattle, WA (SEA)",81.65


In [9]:
rankings.head(3)

,2022 Rank,Airport,2022 Enplaned Passengers,2021 Rank,2021 Enplaned Passengers,Pct. Change 2021-2022
0,1,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...",45.3721,1.0,36.6658,23.745016
1,2,"Dallas/Fort Worth, TX: Dallas/Fort Worth Inter...",35.3294,2.0,30.0006,17.762311
2,3,"Denver, CO: Denver International",33.7499,3.0,28.6285,17.889166


In [10]:
originatingPassengers.head(3)

,Unnamed: 0,Airport,Code,Originating Domestic Passengers
0,1.0,"Los Angeles, CA: Los Angeles International",LAX,1324394
1,2.0,"Denver, CO: Denver International",DEN,1138192
2,3.0,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...",ATL,1088364


In [11]:
airportLOCID.head(3)

,State,City,Airport,LOCID,Ownership,Hub,Role,Current,Year 5,Enplaned,Based,2019-2023 Dev Estimate
0,AK,Adak Island,Adak,ADK,PU,NaN,Basic,GA,GA,2151,0,16166667
1,AK,Akhiok,Akhiok,AKK,PU,NaN,Basic,GA,GA,399,0,8161404
2,AK,Akiachak,Akiachak,Z13,NaN,NaN,Local,CS,GA,3045,0,0


In [12]:
airportPFC.head(3)

,Associated City,State,Airport Name,LOC ID,Hub size,Level,Start Date,Expiration Date,Total PFC Approved (by location)
0,Fairbanks,AK,Fairbanks International,FAI,S,3.0,2000-10-01,2004-04-01,NaN
1,Fairbanks,AK,Fairbanks International,FAI,S,4.5,2004-04-01,2026-10-01,38413252.0
2,Juneau,AK,Juneau International,JNU,N,3.0,1998-10-01,2001-02-01,NaN


In [13]:
airportGrants.head(3)

,State,City,Airport,LOCID,Project Description,Total Amount
0,NaN,NaN,NaN,NaN,NaN,NaN
1,AK,Girdwood,Girdwood,AQY,Seal Apron Pavement Surface/Pavement Joints,504078.0
2,NaN,NaN,NaN,NaN,NaN,NaN


### Cleaning

Now, I want to create datasets for 2022. 

Pandas dataframes all by themselves:
- rankings
- originatingPassengers
- airportLOCID
- airportPFC
- airportPassengersBoarded
- airportGrants 

Other objects:
- airlineArrivalsOnTime
- airportArrivalsOnTimeCurrentMonthly
- airportArrivalsOnTimeHistorical
- airportDepartureOnTimeCurrentMonthly

** need to be accessed with ['key'] to return the pandas dataframe. 

### airportLOCID index table

First thing, I need to use airportLOCID to create index outline to fill my new dataframe. 

I am going to create a set of clean objects that I can merge together somehow in the end. 

In [14]:
columns = ['LOCID', 'Airport','City','State']
cleanLOCID = pd.concat([airportLOCID['LOCID'], airportLOCID['Airport'],airportLOCID['City'],airportLOCID['State']],axis=1)
cleanLOCID.head(10)

,LOCID,Airport,City,State
0,ADK,Adak,Adak Island,AK
1,AKK,Akhiok,Akhiok,AK
2,Z13,Akiachak,Akiachak,AK
3,AKI,Akiak,Akiak,AK
4,7AK,Akutan,Akutan,AK
5,KQA,Akutan,Akutan,AK
6,AUK,Alakanuk,Alakanuk,AK
7,5A8,Aleknagik/New,Aleknagik,AK
8,6A8,Allakaket,Allakaket,AK
9,AFM,Ambler,Ambler,AK


### Using the Rankings dataset

Now, specific to 2022, I am going to pull data from rankings, originatingPassengers, & airportPassengersBoarded.

In [15]:
# clean for bottom headers 
rankings = rankings.head(200)

# Use this to find LOCID for ranked airports 
cleanLOCID[(cleanLOCID['Airport'] == 'Hartsfield-Jackson Atlanta International')]['LOCID']

rankedAirports = rankings['Airport']
rankedAirportNames = rankedAirports.str.extract(r':\s*(.*)$')

rankedLOCID = []
count = 0

for i in range(len(rankedAirportNames)):
    if rankedAirportNames[0][i] == 'Tri Cities WA':
        rankedLOCID += ['TRI'] # need to fix the Tri Cities Airport IATA and I'm not sure where it's wrong
    else:
        try:
            rankedLOCID += [cleanLOCID.loc[cleanLOCID['Airport'] == rankedAirportNames[0][i] , 'LOCID'].iloc[0]]
        except IndexError:
            rankedLOCID += [rankedAirportNames[0][i]]
            count += 1 # This lets me know how many didn't match up from my cleanLOCID DF

rank = rankings['2022 Rank']
conversionfactor = 1000000
numPassengers = rankings['2022 Enplaned Passengers']*conversionfactor

### ChatGPT LOCID cleaning

I  need to clean the rankedLOCID one more time using a different LOCID reference. 

I am going to request the LOCID values for the unknown airports from chatGPT in a .csv file. 

In [16]:
# Use for chatGPT request
for i in range(len(rankedLOCID)):
    if len(rankedLOCID[i]) > 3:
        #print(rankedLOCID[i]) 
        next

extraLOCID = pd.read_csv(partialpath+'extraLOCID.txt')
extraLOCID.head(3)


,Airport,LOCID
0,Dallas/Fort Worth International,DFW
1,John F. Kennedy International,JFK
2,Harry Reid International,LAS


In [17]:
cleanedRankedLOCID = rankedLOCID

for i in range(len(rankedLOCID)):
    if len(cleanedRankedLOCID[i]) > 3:
        temp = extraLOCID.loc[extraLOCID['Airport'] == rankedLOCID[i], 'LOCID'].iloc[0]
        cleanedRankedLOCID[i] = temp

### Combine together for initial dataframe

In [18]:
cleanRank = pd.concat([rank,pd.DataFrame(cleanedRankedLOCID),numPassengers],axis=1)
cleanRank.rename(columns={0: 'Airport'}, inplace=True)
cleanRank.head(10)

,2022 Rank,Airport,2022 Enplaned Passengers
0,1,ATL,45372100.0
1,2,DFW,35329400.0
2,3,DEN,33749900.0
3,4,ORD,33100400.0
4,5,LAX,32271400.0
5,6,JFK,26923300.0
6,7,LAS,25264700.0
7,8,MCO,24431300.0
8,9,MIA,23673000.0
9,10,CLT,23085500.0


### Add more information to this table. 

I am going to work with originatingPassengers. 

originatingPassengers has a column for Code to match up to Airport, then add new column, Originating Domestic Passengers. 

In [19]:
cleanCodes = cleanRank['Airport']
ogDomPassengers = []
originatingPassengers.columns = originatingPassengers.columns.str.strip()

for i in range(len(cleanCodes)):
    if cleanCodes[i] == 'GPI':
        ogDomPassengers += [originatingPassengers.loc[originatingPassengers['Code'] == 'FCA' , 'Originating Domestic Passengers'].iloc[0]]
    else:
        ogDomPassengers += [originatingPassengers.loc[originatingPassengers['Code'] == cleanCodes[i] , 'Originating Domestic Passengers'].iloc[0]]

Add this new variable to the existing table... 

In [20]:
cleanedData = pd.concat([cleanRank, pd.DataFrame(ogDomPassengers)],axis=1)
cleanedData.rename(columns={0: '2022 Originating Domestic Passengers'}, inplace=True)
cleanedData

,2022 Rank,Airport,2022 Enplaned Passengers,2022 Originating Domestic Passengers
0,1,ATL,45372100.0,1088364
1,2,DFW,35329400.0,859264
2,3,DEN,33749900.0,1138192
3,4,ORD,33100400.0,1085583
4,5,LAX,32271400.0,1324394
...,...,...,...,...
195,196,MOT,138700.0,9446
196,197,OAJ,136900.0,9254
197,198,ACK,136500.0,5447
198,199,BET,135600.0,5918


### API requested information

I am using rapidAPI at this link: https://rapidapi.com/proground/api/aviation-reference-data

In [21]:
with open('X-RapidAPI-Key.txt', 'r') as file:
    apiKey = file.read()

Find ICAO codes for each airport in my cleanedData DF...

In [22]:
base_url = "https://aviation-reference-data.p.rapidapi.com/airports/"
endpoints = cleanedData['Airport']

cleanedICAO = [['IATA','ICAO']]

for i in range(len(endpoints)):
    if endpoints[i].strip() == 'GPI': # Glacier Park International
        icao = 'KGPI'
        iata = 'VPA'
    if endpoints[i].strip() == 'VPS': # Eglin AFB Destin Fort Walton Beach
        icao = 'KVPS'
        iata = 'VPS'
        #print(i,endpoints[i]) # 1 = 96
    else:
        url = base_url + endpoints[i].strip()
        headers = {"X-RapidAPI-Key": apiKey,
        "X-RapidAPI-Host": "aviation-reference-data.p.rapidapi.com"}
        
        r = requests.get(url, headers=headers)
        icao = r.json()['icaoCode']
        iata = r.json()['iataCode']

    cleanedICAO += [[iata,icao]]


Clean up the ICAO codes DF so we can merge...

In [23]:
icaoDF = pd.DataFrame(cleanedICAO[1:], columns=cleanedICAO[0])
icaoDF.head(3)

,IATA,ICAO
0,ATL,KATL
1,DFW,KDFW
2,DEN,KDEN


Merge together cleanedData and icaoDF:

In [25]:
cleanedData = pd.merge(cleanedData, icaoDF[['IATA', 'ICAO']], left_on='Airport', right_on='IATA', how='left')
cleanedData = cleanedData.drop('IATA', axis=1)
cleanedData

,2022 Rank,Airport,2022 Enplaned Passengers,2022 Originating Domestic Passengers,ICAO
0,1,ATL,45372100.0,1088364,KATL
1,2,DFW,35329400.0,859264,KDFW
2,3,DEN,33749900.0,1138192,KDEN
3,4,ORD,33100400.0,1085583,KORD
4,5,LAX,32271400.0,1324394,KLAX
...,...,...,...,...,...
195,196,MOT,138700.0,9446,KMOT
196,197,OAJ,136900.0,9254,KOAJ
197,198,ACK,136500.0,5447,KACK
198,199,BET,135600.0,5918,PABE


### Second API information request

I found this API: https://airportdb.io/#

To fetch airport by ICAO code use the following endpoint by including the ICAO code and API token

In [26]:
with open('AirportDBToken.txt', 'r') as file:
    apiToken = file.read()

In [27]:
url_template = "https://airportdb.io/api/v1/airport/{ICAO}?apiToken={apiToken}"
icao = cleanedData['ICAO']

airportDF = pd.DataFrame()

for i in range(len(icao)):
    url = url_template.format(ICAO=icao[i], apiToken=apiToken)
    
    r = requests.get(url)
    if i == 0:
        airportDF = pd.json_normalize(r.json())
    else:
        airportDF = pd.concat([airportDF, pd.json_normalize(r.json())],axis=0)

In [28]:
airportDF.reset_index(drop=True, inplace=True)
airportDF.head(3)

,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality,...,region.id,region.code,region.local_code,region.name,region.continent,region.iso_country,region.wikipedia_link,region.keywords,station.icao_code,station.distance
0,KATL,large_airport,Hartsfield Jackson Atlanta International Airport,33.636700,-84.428101,1026,NA,US,US-GA,Atlanta,...,306086,US-GA,GA,Georgia,NA,US,https://en.wikipedia.org/wiki/Georgia,,KATL,0.0
1,KDFW,large_airport,Dallas Fort Worth International Airport,32.896801,-97.038002,607,NA,US,US-TX,Dallas-Fort Worth,...,306119,US-TX,TX,Texas,NA,US,https://en.wikipedia.org/wiki/Texas,,KDFW,0.0
2,KDEN,large_airport,Denver International Airport,39.861698,-104.672997,5431,NA,US,US-CO,Denver,...,306081,US-CO,CO,Colorado,NA,US,https://en.wikipedia.org/wiki/Colorado,,KDEN,0.0


### Editing loop for missing vals 

- instead of using concat, use merge on IATA or ICAO

In [29]:
#for index, row in airportDF.iterrows():
#    print(row)

When I reviewed the dataset log/lat, I realized there is an error for MSO - Missoula Montana Airport. 

I am trying to figure out where it's going wrong. Down below, I printed the results from the API for KMSO, and it's correct, so something must be wrong with the merge?

In [30]:
#condition = airportDF['ident'] == 'KMSO'
#print(pd.DataFrame(airportDF[condition]))

### Clean Nulls 

In reviewing my table, I realized there are some null values that I want to take care of.

Airport *GPI* is Glacier Park International Airport, which has 2 codes: GPI and FCA. 

I need to update it from GPI to FCA in order for the merge on airportStats to work correctly. 

Since I added in safety measures in previous loops & chunks for GPI before I updated it here, the information will all be correct, regardless. 

In [31]:
cleanedData['Airport'] = cleanedData['Airport'].replace('GPI', 'FCA')

### Add data to my table...

In [32]:
#cleanedData = pd.concat([cleanedData, pd.DataFrame(airportDF['latitude_deg'])],axis=1)
#cleanedData = pd.concat([cleanedData, pd.DataFrame(airportDF['longitude_deg'])],axis=1)
#cleanedData = pd.concat([cleanedData, pd.DataFrame(airportDF['elevation_ft'])],axis=1)
#cleanedData.rename(columns={'latitude_deg': 'Latitude', 'longitude_deg': 'Longitude', 'elevation_ft': 'Elevation'}, inplace=True)
#cleanedData.head(5)

In [33]:
cleanDF = pd.merge(cleanedData, airportDF[['iata_code', 'icao_code', 'name', 'latitude_deg', 'longitude_deg', 'elevation_ft', 'region.name']], left_on='ICAO', right_on='icao_code', how='left')
cleanDF.reset_index(drop=True, inplace=True)

In [34]:
# Compare Airport in tempcleanDF to Airport in cleanedData to make sure the ordered rows didn't change
#for i in range(len(cleanDF)):
#    check = cleanDF['Airport'][i] == cleanedData['Airport'][i]
#    if check == False:
#        print(cleanDF['Airport'][i], cleanedData['Airport'][i])

cleanDF = cleanDF.drop('iata_code', axis=1)
cleanDF = cleanDF.drop('icao_code', axis=1)
cleanDF.rename(columns={'Airport': 'IATA'}, inplace=True)
cleanDF.rename(columns={'latitude_deg': 'Latitude', 
                        'longitude_deg': 'Longitude', 
                        'elevation_ft': 'Elevation', 
                        'region.name': 'State',
                        'name': 'Airport'}, inplace=True)

cleanDF.head(10)

,2022 Rank,IATA,2022 Enplaned Passengers,2022 Originating Domestic Passengers,ICAO,Airport,Latitude,Longitude,Elevation,State
0,1,ATL,45372100.0,1088364,KATL,Hartsfield Jackson Atlanta International Airport,33.636700,-84.428101,1026,Georgia
1,2,DFW,35329400.0,859264,KDFW,Dallas Fort Worth International Airport,32.896801,-97.038002,607,Texas
2,3,DEN,33749900.0,1138192,KDEN,Denver International Airport,39.861698,-104.672997,5431,Colorado
3,4,ORD,33100400.0,1085583,KORD,Chicago O'Hare International Airport,41.978600,-87.904800,672,Illinois
4,5,LAX,32271400.0,1324394,KLAX,Los Angeles International Airport,33.942501,-118.407997,125,California
5,6,JFK,26923300.0,763958,KJFK,John F Kennedy International Airport,40.639801,-73.778900,13,New York
6,7,LAS,25264700.0,829770,KLAS,McCarran International Airport,36.080101,-115.152000,2181,Nevada
7,8,MCO,24431300.0,925518,KMCO,Orlando International Airport,28.429399,-81.308998,96,Florida
8,9,MIA,23673000.0,537910,KMIA,Miami International Airport,25.793200,-80.290604,8,Florida
9,10,CLT,23085500.0,409225,KCLT,Charlotte Douglas International Airport,35.214001,-80.943100,748,North Carolina


### Cleaning up long/lat errors 

Look at individual rows based on some condition

In [35]:
#condition = tempcleanDF['Airport'] == 'FCA'

#print(pd.DataFrame(tempcleanDF[condition]))

### Adding more columns from AirportStats.txt

I like this table so far and there is a lot to work with, but not many interesting quantitative columns to compare. I am going to try to find some more information to merge into my table. 

I found this website: https://www.stratosjets.com/blog/us-airport-rankings/ with an updated 2022 table of US airports and for each...  
- Flights Per Year	
- Delays %	
- Canceled %	
- Avg. Delay (Mins)	
- Security Delay (Mins)

I copied the data into a .txt file named: AirportStats.txt

In [36]:
airportStats = pd.read_csv(partialpath+'AirportStats.txt',sep="\t")
airportStats.head(3)

,Overall Rank,State,Code,Airport,Flights Per Year,Delays %,Canceled %,Avg. Delay (Mins),Security Delay (Mins)
0,1,HI,ITO,Hilo International,"5,595",5.8,0.2,3.7,0.00
1,2,HI,OGG,Kahului Airport,"24,834",10.7,0.4,8.5,0.02
2,3,UT,SLC,Salt Lake City International,"105,043",13.4,0.4,10.2,0.02



Now, I want to grab the last 5 columns for each airport in my cleanedData DF. 

In [37]:
finalDF = pd.merge(cleanDF, airportStats[['Code', 'Flights Per Year', 'Delays %', 'Canceled %', 'Avg. Delay (Mins)', 'Security Delay (Mins)']], left_on='IATA', right_on='Code', how='left')
finalDF = finalDF.drop('Code', axis=1)
finalDF['Delays %'] = finalDF['Delays %'] / 100
finalDF['Canceled %'] = finalDF['Canceled %'] / 100
columnOrder = ['2022 Rank', 'Airport', 'IATA', 'ICAO', '2022 Enplaned Passengers', 
               '2022 Originating Domestic Passengers', 'Flights Per Year', 'Delays %', 
               'Canceled %', 'Avg. Delay (Mins)', 'Security Delay (Mins)', 'Latitude', 
               'Longitude', 'State', 'Elevation']
finalDF = finalDF[columnOrder]
finalDF.rename(columns={'Delays %': 'Delays', 'Canceled %': 'Canceled'}, inplace=True)
finalDF

,2022 Rank,Airport,IATA,ICAO,2022 Enplaned Passengers,2022 Originating Domestic Passengers,Flights Per Year,Delays,Canceled,Avg. Delay (Mins),Security Delay (Mins),Latitude,Longitude,State,Elevation
0,1,Hartsfield Jackson Atlanta International Airport,ATL,KATL,45372100.0,1088364,"362,713",0.168,0.007,11.1,0.06,33.636700,-84.428101,Georgia,1026
1,2,Dallas Fort Worth International Airport,DFW,KDFW,35329400.0,859264,"278,828",0.223,0.028,15.7,0.03,32.896801,-97.038002,Texas,607
2,3,Denver International Airport,DEN,KDEN,33749900.0,1138192,"230,693",0.226,0.020,16.0,0.04,39.861698,-104.672997,Colorado,5431
3,4,Chicago O'Hare International Airport,ORD,KORD,33100400.0,1085583,"312,159",0.228,0.033,17.6,0.02,41.978600,-87.904800,Illinois,672
4,5,Los Angeles International Airport,LAX,KLAX,32271400.0,1324394,"201,287",0.185,0.013,12.7,0.15,33.942501,-118.407997,California,125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,196,Minot International Airport,MOT,KMOT,138700.0,9446,"2,070",0.132,0.016,13.9,0.17,48.259399,-101.279999,North Dakota,1716
196,197,Albert J Ellis Airport,OAJ,KOAJ,136900.0,9254,"2,611",0.155,0.014,12.3,0.01,34.829201,-77.612099,North Carolina,94
197,198,Nantucket Memorial Airport,ACK,KACK,136500.0,5447,"1,195",0.268,0.031,33.3,0.24,41.253101,-70.060204,Massachusetts,47
198,199,Bethel Airport,BET,PABE,135600.0,5918,759,0.129,0.029,8.3,6.77,60.779800,-161.837997,Alaska,126


### Drop Duplicates

My finalDF is 203 rows long, which is too many. Below I find where the duplicates are...

In [38]:
for i in range(len(finalDF['IATA'])):
    if finalDF['IATA'].duplicated()[i] == True:
        print(finalDF['IATA'][i])

In [39]:
finalDF.drop_duplicates(subset=['IATA'], keep='first', inplace=True, ignore_index=True)
finalDF

,2022 Rank,Airport,IATA,ICAO,2022 Enplaned Passengers,2022 Originating Domestic Passengers,Flights Per Year,Delays,Canceled,Avg. Delay (Mins),Security Delay (Mins),Latitude,Longitude,State,Elevation
0,1,Hartsfield Jackson Atlanta International Airport,ATL,KATL,45372100.0,1088364,"362,713",0.168,0.007,11.1,0.06,33.636700,-84.428101,Georgia,1026
1,2,Dallas Fort Worth International Airport,DFW,KDFW,35329400.0,859264,"278,828",0.223,0.028,15.7,0.03,32.896801,-97.038002,Texas,607
2,3,Denver International Airport,DEN,KDEN,33749900.0,1138192,"230,693",0.226,0.020,16.0,0.04,39.861698,-104.672997,Colorado,5431
3,4,Chicago O'Hare International Airport,ORD,KORD,33100400.0,1085583,"312,159",0.228,0.033,17.6,0.02,41.978600,-87.904800,Illinois,672
4,5,Los Angeles International Airport,LAX,KLAX,32271400.0,1324394,"201,287",0.185,0.013,12.7,0.15,33.942501,-118.407997,California,125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,196,Minot International Airport,MOT,KMOT,138700.0,9446,"2,070",0.132,0.016,13.9,0.17,48.259399,-101.279999,North Dakota,1716
196,197,Albert J Ellis Airport,OAJ,KOAJ,136900.0,9254,"2,611",0.155,0.014,12.3,0.01,34.829201,-77.612099,North Carolina,94
197,198,Nantucket Memorial Airport,ACK,KACK,136500.0,5447,"1,195",0.268,0.031,33.3,0.24,41.253101,-70.060204,Massachusetts,47
198,199,Bethel Airport,BET,PABE,135600.0,5918,759,0.129,0.029,8.3,6.77,60.779800,-161.837997,Alaska,126


In [40]:
#for index, row in finalDF.iterrows():
#    print(row)

### Export

Save finalDF to it's own .csv file...

In [41]:
finalDF.to_csv('2022Airports.csv', index=False)